In [1]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from torch.utils.data import TensorDataset
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,confusion_matrix
from utils.function import prep_data
from models.de_OPPO import DAE
from models.activity_recognition import *
# from models.activity_recognition_OPPO import get_eval_model
from models.dis_z import DIS_Z
import copy
import sklearn.metrics as metrics
import os
#from tqdm import tqdm

In [2]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 100
        self.nz = 200
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = 'data/experiments/DE_OPPO1'
        self.commit = 'eval'
        self.alpha = 1.0
        # self.sigma = 0.35
        self.M = 5
        self.loss = 'MSE' #'BCE'
        self.loadDAE = False
        self.loadSVM = False    
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 2
        self.multimodalZ = False
        self.window_len = 512
        self.stride_len = 20
        self.act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
        self.imSize = 64
        # self.sigma = [60, 80]
        self.sigma=0.05
        # self.sigma = [6, 10]
        self.cuda_id = 2
        self.random_seed = 2
        self.train_split = 0.8

        #self.corr= 'ZeroMask' # options: Gaussian, ZeroMask, ConsecutiveZeros
        # if self.corr == 'ConsecutiveZeros':
        #     self.sigma = [40, 80] # [lambda_corr, lambda_norm]

        # path for corruption "consecutive interval"sigma [60,80]
        self.dae_model_loc = "data/experiments/DAE_EVAL_Noise/Ex_12" 

        self.ar_model_loc = "data/opportunity_ConvAttn.pt"


In [3]:
gpu_id = 3

if gpu_id>=0:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    cuda_id = "cuda:" + str(0)  # cuda:2

device = torch.device(cuda_id if torch.cuda.is_available() else "cpu")
print("Device:", device)
if (torch.cuda.is_available()):
    torch.cuda.set_device(cuda_id)
    print("Current GPU ID:", torch.cuda.current_device())

Device: cuda:0
Current GPU ID: 0


In [4]:
def build_dis(args, dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = args.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ


def analysis(args, dae, testDataset, X, num_classes):
    # Prepare testdata set, drop the last incomplete batch
    test_x = torch.zeros(len(testDataset), X.shape[1], X.shape[2], X.shape[3])
    test_labels = torch.zeros(len(testDataset), num_classes)
    for test_id in range(len(testDataset)):
        test_labels[test_id] = testDataset[test_id][1]
        test_x[test_id] = testDataset[test_id][0]
    print(test_x.shape)

    # Corrupt dataset
    corr_test_x = dae.corrupt(test_x)
    
    mean_fill_test = interpolation_meanfilling(corr_test_x)
    linear_interp_test = linear_interpolation(corr_test_x)
# recon_test -> synthesize the entire dataset
    z = dae.encode(corr_test_x)
    recon_test = dae.decode(z)

    # breakpoint()
# reconstruct filling testset (fill missing values only, not valid for noisy data)
    recon_fill_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
    np.copyto(recon_fill_test, recon_test.detach().numpy(), where = recon_fill_test==0)
    recon_fill_test = torch.from_numpy(recon_fill_test)

    # Reconstruct testset
    # corr_test_dataset = TensorDataset(outputs.permute(0, 2, 1))
    #recon_test = torch.from_numpy(recon_test)#.permute(1, 2, 0)
    
    # test_x = test_x.reshape(test_x.shape[0],test_x.shape[2],test_x.shape[3])
    # corr_test_x = corr_test_x.reshape(corr_test_x.shape[0],corr_test_x.shape[2],corr_test_x.shape[3])
    # recon_test = recon_test.reshape(recon_test.shape[0],recon_test.shape[2],recon_test.shape[3])
    # recon_fill_test = recon_fill_test.reshape(recon_fill_test.shape[0],recon_fill_test.shape[2],recon_fill_test.shape[3])
    # mean_fill_test = mean_fill_test.reshape(mean_fill_test.shape[0],mean_fill_test.shape[2],mean_fill_test.shape[3])
    # test_x = test_x.permute(0,1,3,2)
    # corr_test_x = corr_test_x.permute(0,1,3,2)
    # recon_test = recon_test.permute(0,1,3,2)
    # recon_fill_test = recon_fill_test.permute(0,1,3,2)
    # mean_fill_test = mean_fill_test.permute(0,1,3,2)


    raw_test_dataset = TensorDataset(test_x, test_labels)
    corr_test_dataset = TensorDataset(corr_test_x, test_labels)
    recon_test_dataset = TensorDataset(recon_test, test_labels)
    recon_fill_test_dataset = TensorDataset(recon_fill_test, test_labels)
    mean_fill_test_dataset = TensorDataset(mean_fill_test, test_labels)
    linear_interp_test_dataset = TensorDataset(linear_interp_test, test_labels)

    return test_x, corr_test_x, recon_test, recon_fill_test, mean_fill_test, linear_interp_test, raw_test_dataset, corr_test_dataset, recon_test_dataset, recon_fill_test_dataset, mean_fill_test_dataset,linear_interp_test_dataset

In [5]:
from utils.sliding_window import sliding_window
import pickle as cp

In [6]:
# Number of Sensor Channels used in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Number of classes in which data is classified (or to be classified).
NUM_CLASSES = 5

# Length of the sliding window used to segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Steps of the sliding window used in segmenting the data.
SLIDING_WINDOW_STEP = 12

act_labels_txt = ['std', 'wlk', 'sit', 'lie', 'null']

In [7]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading Data...")
X_train, y_train, X_test, y_test = load_dataset('../../../../../data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
X_test = np.transpose(X_test, (0, 2, 1))
X_test= X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]) # convert list to numpy array
X_test.shape

Loading Data...
 ..from file ../../../../../data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


(9894, 1, 113, 24)

In [8]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
X_train = np.transpose(X_train, (0, 2, 1))
X_train= X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]) # convert list to numpy array

X_train.shape

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 1, 113, 24)

In [9]:
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [12]:
def interpolation_meanfilling(corr_test_x):
    mean_fill_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
    for i in range(mean_fill_test.shape[0]):
        for j in range(mean_fill_test.shape[2]):
            if np.count_nonzero(mean_fill_test[i,:,j,:]) == 0:  
                ch_mean = 0
            else:
             #ch_mean = np.sum(mean_fill_test[i][0][j]) / np.count_nonzero(mean_fill_test[i][0][j])
                ch_mean = np.sum(mean_fill_test[i,:,j,:]) / np.count_nonzero(mean_fill_test[i,:,j,:])
                mean_fill_test[i,:,j,:][mean_fill_test[i,:,j,:] == 0] = ch_mean
    mean_fill_test = torch.from_numpy(mean_fill_test)
    return mean_fill_test

def linear_interpolation(corr_test_x):
    linear_interp_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
    # corr_text_x shape is 18944, 1, 27, 171
    # expected shape 18944, 171, 27
    # linear_interp_test = linear_interp_test.reshape(-1, 171,27)
    linear_interp_test = linear_interp_test.reshape(-1, linear_interp_test.shape[3],linear_interp_test.shape[2])
    for i in range(linear_interp_test.shape[0]):
        for j in range(linear_interp_test.shape[2]):
            if np.count_nonzero(linear_interp_test[i,:,j]) == 0: # when all data points in this channel are missing
                linear_interp_test[i, :, j] = 0.0
            else:
                idxs = np.arange(linear_interp_test.shape[1]) # indexes of all the samples
                zero_filter = linear_interp_test[i,:,j] == 0 # index filter for zero values
                zero_idxs = idxs[zero_filter] # indexes for zero values
                non_zero_idxs = idxs[~zero_filter] # xp, indexes for non-zero values
                non_zero_vals = linear_interp_test[i, ~zero_filter,j] # fp, non-zero values
                interp_vals = np.interp(zero_idxs, non_zero_idxs, non_zero_vals) # interpolated values
                linear_interp_test[i,zero_idxs,j] = interp_vals # fill interpolated values to the corrupted signal
    linear_interp_test = torch.from_numpy(linear_interp_test)
    linear_interp_test = torch.reshape(linear_interp_test, (-1, 1, linear_interp_test.shape[2], linear_interp_test.shape[1]))
    return linear_interp_test


def evaluate_rmse(corr_test_x, recon_test, recon_fill_test,mean_fill_test,linear_interp_test,test_x):
    corr_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), corr_test_x.reshape(corr_test_x.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Corr RMSE:\n' + str(corr_rms))

    recon_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), recon_test.reshape(recon_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Recon RMSE:\n' + str(recon_rms))

    recon_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), recon_fill_test.reshape(recon_fill_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Recon fill RMSE:\n' + str(recon_fill_rms))

    mean_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().detach().numpy(), mean_fill_test.reshape(mean_fill_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Mean Fill RMSE:\n' + str(mean_fill_rms))
    
    linear_interp_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1), linear_interp_test.reshape(linear_interp_test.shape[0],-1).cpu().detach().numpy(), squared=False)
    print('Linear Interpolation RMSE:\n' + str(linear_interp_rms))
    return 
#def plot(test_x, corr_test_x, recon_test)

def plot(test_x, corr_test_x, recon_test, recon_fill_test,mean_fill_test, linear_interp_test):
    plt.imshow(test_x[0][0].detach())
    plt.title('Raw Data')
   # plt.subplot(4,1,1)
    plt.savefig(join(args.dae_model_loc, 'raw.png'))

    plt.imshow(corr_test_x[0][0].detach())
    plt.title('Corrupted')
   # plt.subplot(4,1,2)
    plt.savefig(join(args.dae_model_loc, 'corr.png'))

    plt.imshow(recon_test[0][0].detach())
    plt.title('Reconstructed')
   # plt.subplot(4,1,3)
    plt.savefig(join(args.dae_model_loc, 'reconstructed.png'))

    plt.imshow(recon_fill_test[0][0].detach())
    plt.title('Reconstructed fill')
   # plt.subplot(4,1,3)
    plt.savefig(join(args.dae_model_loc, 'rec_fill.png'))

    plt.imshow(mean_fill_test[0][0].detach())
    plt.title('Mean Fill')
   # plt.subplot(4,1,4)
    plt.savefig(join(args.dae_model_loc, 'mean.png'))
    
    plt.imshow(linear_interp_test[0][0].detach())
    plt.title('Linear Interp')
   # plt.subplot(4,1,4)
    plt.savefig(join(args.dae_model_loc, 'linear.png'))
    
    return

def test_dae(args, path, trainLoader, testLoader):
    pass


def test_activity_recognition(args, path, trainLoader, testLoader):
    pass


def calculate_combined_accuracy(args, test_loader, sigma):
    # device = torch.device(args.cuda_id if torch.cuda.is_available() else "cpu")
    
    args.n_sensor_channels = 113
    args.len_seq = 24
    args.num_classes=5
    ar = get_eval_model(n_sensor_channels=args.n_sensor_channels, len_seq=args.len_seq, num_classes=args.num_classes, model_path=args.ar_model_loc) #n_sensor_channels, len_seq, num_classes, model_path
    ar = ar.to(device)
    
    correct = 0
    total = 0
    total_true = []
    total_pred = []

    with torch.no_grad():   
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)  
            # print(images.shape)
            
            outputs = ar(images)

            # _, predicted = torch.max(outputs.data, 1)
            predicted = torch.argmax(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
            
            total_pred = total_pred + predicted.cpu().numpy().tolist()
            total_true = total_true + (torch.argmax(labels, dim=1).cpu().numpy().tolist())
            
    print(f'Test Accuracy: \n{100.0 * correct / total} ')
    
    # print(" | ".join(act_labels_txt))
    conf_mat = confusion_matrix(y_true = total_true, y_pred = total_pred)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print(np.array(conf_mat).round(3) * 100)  
    f1_score = metrics.f1_score(y_true = total_true, y_pred = total_pred, average='weighted')
    print('F1 score:\n', f1_score)
    print('')
    return 


In [13]:
if __name__ == "__main__":
    args = Args()
    if args.multimodalZ:
        args.nz = 2
        
    args.num_classes = 5
    # random.seed(args.random_seed)
    # np.random.seed(args.random_seed)
    # torch.manual_seed(args.random_seed)

    X = torch.from_numpy(X_test[:9856,:,:])
    labels = torch.from_numpy(y_test[:9856,:])
    # X, labels = prepare_data(args)
    # trainDataset, testDataset, trainLoader, testLoader = prepare_dataloaders(args, X, labels)
    
    testDataset = TensorDataset(X, labels)


    testLoader = torch.utils.data.DataLoader(testDataset,
        batch_size=args.batchSize, shuffle=False)    
    
    dae = DAE(nz=args.nz, imSize=args.imSize, fSize=args.fSize, sigma=args.sigma, multimodalZ=args.multimodalZ)
    dae.load_params(args.dae_model_loc)
    # if dae.useCUDA:
    #     torch.cuda.set_device(args.gpuNo)
    #     # torch.cuda.set_device(args.gpuNo)
    #     dae.cuda()
    #     # dis.cuda()

    # test_dae(args, dae_path, trainLoader, testLoader)
    # test_activity_recognition(args, ar_path, trainLoader, testLoader)

    #acc = calculate_combined_accuracy(args, testLoader, sigma=0.5)
    test_x, corr_test_x, recon_test, recon_fill_test, mean_fill_test, linear_interp_test, raw_test_dataset, corr_test_dataset, recon_test_dataset, recon_fill_test_dataset, mean_fill_test_dataset, linear_interp_test_dataset = analysis(args, dae, testDataset, X_test, args.num_classes)
    # test_x, corr_test_x, recon_test, recon_fill_test, mean_fill_test, raw_test_dataset, corr_test_dataset, recon_test_dataset, recon_fill_test_dataset, mean_fill_test_dataset = analysis(dae, testDataset, labels)
    #corr_test_x, test_x, recon_test, raw_test_dataset, corr_test_dataset, recon_test_dataset = analysis(dae, testDataset, labels)

    # test_x = test_x.permute(0,1,3,2)
    # corr_test_x = corr_test_x.permute(0,1,3,2)
    # recon_test = recon_test.permute(0,1,3,2)
    # recon_fill_test = recon_fill_test.permute(0,1,3,2)
    # mean_fill_test = mean_fill_test.permute(0,1,3,2)
    # linear_interp_test = linear_interp_test.permute(0,1,3,2)

    print("Raw testset:")
    raw_test_loader = torch.utils.data.DataLoader(raw_test_dataset,
    batch_size= args.batchSize, shuffle=False)
    calculate_combined_accuracy(args, raw_test_loader, sigma=args.sigma)

    print("Corrupted testset:")
    corr_test_loader = torch.utils.data.DataLoader(corr_test_dataset,
    batch_size= args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,corr_test_loader, sigma=args.sigma)

    print("Reconstructed testset:")
    recon_test_loader = torch.utils.data.DataLoader(recon_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,recon_test_loader, sigma=args.sigma)


    print("Reconstructed fill testset:")
    recon_fill_test_loader = torch.utils.data.DataLoader(recon_fill_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,recon_fill_test_loader, sigma=args.sigma)


    print("Mean Fill testset")
    mean_fill_test_loader = torch.utils.data.DataLoader(mean_fill_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,mean_fill_test_loader, sigma=args.sigma)
    
    print("Linear Interpolation testset:")
    linear_interp_test_loader = torch.utils.data.DataLoader(linear_interp_test_dataset,
    batch_size=args.batchSize, shuffle=False)
    calculate_combined_accuracy(args,linear_interp_test_loader, sigma=args.sigma)

    evaluate_rmse(corr_test_x, recon_test,recon_fill_test, mean_fill_test, linear_interp_test,test_x)
    plot(test_x, corr_test_x, recon_test,recon_fill_test, mean_fill_test, linear_interp_test)


loading params...
torch.Size([9856, 1, 113, 24])
Raw testset:
Test Accuracy: 
89.21469155844156 
[[79.   4.9 13.8  1.4  0.8]
 [ 0.8 93.8  4.3  1.1  0. ]
 [ 2.9 14.3 82.4  0.4  0. ]
 [ 0.6  0.6  0.  98.6  0.1]
 [ 1.5  0.   0.2  3.  95.2]]
F1 score:
 0.8912435729751317

Corrupted testset:
Test Accuracy: 
84.93303571428571 
[[75.9  2.3 19.1  2.   0.6]
 [ 1.1 79.  18.6  1.2  0. ]
 [ 3.3  9.2 87.1  0.3  0. ]
 [ 0.8  0.5  0.  98.5  0.1]
 [ 3.   0.   0.6  2.6 93.7]]
F1 score:
 0.8515978830890847

Reconstructed testset:
Test Accuracy: 
88.58563311688312 
[[78.7  5.  14.1  1.4  0.7]
 [ 0.8 94.2  3.5  1.5  0. ]
 [ 3.3 17.  79.3  0.4  0. ]
 [ 0.7  0.6  0.  98.6  0.1]
 [ 1.1  0.   0.2  2.8 95.9]]
F1 score:
 0.8845829896895254

Reconstructed fill testset:
Test Accuracy: 
84.93303571428571 
[[75.9  2.3 19.1  2.   0.6]
 [ 1.1 79.  18.6  1.2  0. ]
 [ 3.3  9.2 87.1  0.3  0. ]
 [ 0.8  0.5  0.  98.5  0.1]
 [ 3.   0.   0.6  2.6 93.7]]
F1 score:
 0.8515978830890847

Mean Fill testset
Test Accuracy: 
84.933